# PJN - Lab 10 - Podsumowanie

#### Mikołaj Stefański

## Ładowanie modułów

In [1]:
import sys
import os
import json
import pandas as pd

sys.path.append(os.getcwd())

from rag.metadata.ner import NERExtractor
from rag.retrieval.search_engine import hybrid_search
from rag.reasoning.agent_brain import extract_search_params

print("Moduły załadowane poprawnie.")

Moduły załadowane poprawnie.


## Etap 1 — Ekstrakcja encji nazwanych (NER) i dat

### Analiza jakości NER

In [2]:
ENRICHED_FILE = "data/culturax_enriched.jsonl"
samples = []

print(f"Analiza pliku: {ENRICHED_FILE}")
try:
    with open(ENRICHED_FILE, 'r', encoding='utf-16') as f:
        for i, line in enumerate(f):
            if i >= 1000: break
            doc = json.loads(line)
            if doc.get('named_entities') or doc.get('years'):
                samples.append(doc)
                if len(samples) >= 5: break

    df = pd.DataFrame(samples)[['text', 'named_entities', 'years']]
    df['text'] = df['text'].apply(lambda x: x[:80] + "...")
    display(df)

except FileNotFoundError:
    print("Nie znaleziono pliku enriched.")

Analiza pliku: data/culturax_enriched.jsonl


,text,named_entities,years
0,Julie Wang 50PCS Big White Clear Self-Seal Zip...,[Duży Rozmiar Biały Jasny Self - Seal Zipper P...,[]
1,"Uwaga:1 cm=10 mm,1 mm=0,0393 cala,1 cal=25,4 m...",[Formy Do Odlewania Wi Moda Biżuteria Opakowan...,[]
2,"Nie ulega wątpliwości, że dalsze doskonalenie ...",[Polsce],[]
3,Kolejnym etapem kampanii „Moja biblioteka” był...,[Moja biblioteka],[]
4,Podsumowanie Przez ostatnie 10 latach nastąpił...,[Podsumowanie],[]


#### Wyniki

1. Szum i false positives - wiersze 0 i 1

    Wiersze 0 i 1 to typowe śmieci z datasetu CulturaX - dane z common crawl, często e-commerce typu AliExpress. Model NER widzi długi ciąg słów pisanych wielką literą  i błędnie interpretuje to jako nazwę własną organizację lub produkt - model jest wrażliwy na styl pisowni title case w ofertach sprzedażowych.

2. Sukces - wiersz 2

    To jest dowód, że model działa poprawnie. Mimo że słowo jest w innej formie fleksyjnej (miejscownik: w Polsce), model je wyłapał.

3. Niejednoznaczność - wiersze 3 i 4

    "Moja biblioteka" to nazwa kampanii; widać cudzysłów we fragmencie tekstu. Model poprawnie uznał to za nazwę własną, choć technicznie to tytuł wydarzenia/akcji. Model prawdopodobnie uznał to za encję, ponieważ słowo występuje na początku nagłówka/zdania i jest pisane wielką literą. To typowy błąd capitalization bias.

## 4.1

In [4]:
import json
import random
import pandas as pd

ENRICHED_FILE = "data/culturax_enriched.jsonl"

try:
    with open(ENRICHED_FILE, 'r', encoding='utf-16') as f:
        lines = f.readlines()

    samples = random.sample(lines, 25)
    data_for_display = []

    for line in samples:
        doc = json.loads(line)
        data_for_display.append({
            "Tekst": doc.get('text', '')[:100] + "...",
            "Wykryte Encje": doc.get('named_entities', [])
        })

    df = pd.DataFrame(data_for_display)
    display(df)

except FileNotFoundError:
    print("Najpierw uruchom skrypt enrich_data.py, aby stworzyć plik z danymi.")

,Tekst,Wykryte Encje
0,"Hello Web Admin, I noticed that your On-Page S...","[Web Admin, Page]"
1,Krótko o parowych łaźniachWokół każdego domu- ...,[Które Zmienią Wszystko Co Myślałeś O Tym Jak ...
2,Jest wiceprezesem i dyrektorem generalnym mark...,"[Artemide, Danese Milano]"
3,Dokonując zakupów w naszym sklepie macie gwara...,[]
4,Pierwszy wpis związany z przodkami jaki znajdu...,"[Kata, ##ziubów, Przedborza, Sobol, Tomasz]"
5,"Niezależnie od tego, jak piękne hasła będzie p...",[]
6,"Mowa tu o tym, że jeśli ten konkretny producen...",[]
7,Raport z 17 maja 2 167 nowych zakażeń koronawi...,[]
8,Ryczałt ewidencjonowany opłaca się według staw...,[##y]
9,autor opinii: Anna Lebioda Książeczka godna pr...,"[Anna Lebioda, Katarzyna Wołcyrz Niepozor]"


#### Wnioski z analizy modelu NER

Analiza 25 losowych fragmentów z korpusu `CulturaX` przy użyciu modelu `Babelscape/wikineural-multilingual-ner` przyniosła następujące obserwacje:

1. Rozpoznawane typy encji
    Model poprawnie identyfikuje kluczowe kategorie semantyczne:
    - **PER (Osoby):** wykrywa zarówno imiona, jak i pełne nazwiska np. "Anna Lebioda", "Katarzyna Wołcyrz Niepozor", "Tomasz", "Marek Wasiluk".
    - **ORG (Organizacje):** rozpoznaje firmy i instytucje np. "Techland", "Artemide", "Danese Milano", "Urzędem Marszałkowskim Województwa Opolskiego".
    - **LOC (Lokalizacje):** wykrywa nazwy geograficzne np. "Przedborza".
    - **MISC / Inne:** rozpoznaje nazwy produktów, technologii lub grup wyznaniowych np. "RoomPerfect", "IPS", "Świadkowie Jehowy", "Biblia".

2. Typowe błędy i problemy
    - **Capitalization Bias:** model ma tendencję do błędnego oznaczania długich ciągów wyrazów jako encji, jeśli są pisane w stylu nagłówkowym. Widać to w próbce nr 1: "Które Zmienią Wszystko Co Myślałeś...", błędnie zaklasyfikowane jako encja.
    - **Artefakty tokenizacji:** w próbce nr 4 pojawia się encja "##ziubów". Znak `##` jest charakterystyczny dla tokenizatorów modeli BERT i oznacza fragment słowa. Wskazuje to na problem ze sklejeniem tokenów po stronie pipeline'u w rzadkich przypadkach.
    - **Szum językowy:** próbka nr 0 jest w języku angielskim "Web Admin", co model próbował zinterpretować jako osobę/rolę, mimo że kontekst jest polski.

3.  Fleksja i odmiana
    Model radzi sobie z polską deklinacją bardzo dobrze, co jest jego największą przewagą nad metodami RegExp:
    - **Próbka 11:** "Urzędem Marszałkowskim Województwa Opolskiego" – poprawnie rozpoznano długą nazwę w nadrzędniku (kim/czym?).
    - **Próbka 15:** "Marka Wasiluka" – poprawnie rozpoznano imię i nazwisko w dopełniaczu (kogo? czego?), wiążąc to z kategorią PER.
    - **Próbka 4:** "Przedborza" – poprawnie rozpoznano nazwę miasta w dopełniaczu.

### 4.2

In [1]:
import pandas as pd
from rag.metadata.ner import NERExtractor
from rag.metadata.date_extractor_regex_llm import DateExtractorHybrid

ner_model = NERExtractor()
date_hybrid = DateExtractorHybrid()

test_sentences = [
    "Urodziłem się 12.05.1990 w Warszawie, a w 2020 roku przeprowadziłem się.",
    "Projekt trwał od 2010 do 2015 roku i zakończył się sukcesem.",
    "Spotkanie odbędzie się 2024-12-24. Pamiętaj o terminie!",
    "Lata 90-te były czasem przemian w Polsce."
]

results = []

for text in test_sentences:
    res_regex = date_hybrid.extract_regex(text)
    
    res_ner = ner_model.extract(text)
    
    res_llm = date_hybrid.extract_llm(text)
    
    results.append({
        "Tekst": text,
        "1. RegExp": res_regex,
        "2. NER (Wynik)": res_ner,
        "3. LLM": res_llm
    })

pd.set_option('display.max_colwidth', None)
display(pd.DataFrame(results))

Ładowanie modelu NER: Babelscape/wikineural-multilingual-ner.


Device set to use cpu


Model NER gotowy.


,Tekst,1. RegExp,2. NER (Wynik),3. LLM
0,"Urodziłem się 12.05.1990 w Warszawie, a w 2020 roku przeprowadziłem się.","{'years': [1990, 2020], 'dates': ['12.05.1990']}",[Warszawie],"{'dates': ['12.05.1990', '2020'], 'years': [1990, 2020], 'ranges': ['w Warszawie', 'przeprowadziłem się']}"
1,Projekt trwał od 2010 do 2015 roku i zakończył się sukcesem.,"{'years': [2010, 2015], 'dates': []}",[],"{'dates': ['2010', '2015'], 'years': [2010, 2015], 'ranges': ['od 2010 do 2015 roku']}"
2,Spotkanie odbędzie się 2024-12-24. Pamiętaj o terminie!,"{'years': [2024], 'dates': ['2024-12-24']}",[Pamiętaj o terminie!],"{'dates': ['2024-12-24'], 'years': [2024], 'ranges': ['termin']}"
3,Lata 90-te były czasem przemian w Polsce.,"{'years': [], 'dates': []}",[Polsce],"{'dates': [], 'years': [90], 'ranges': ['lata 90-te']}"


#### Wyniki

1.  RegExp
    - **Zalety:** bezbłędnie wyciąga daty w sztywnych formatach (ISO `2024-12-24`, PL `12.05.1990`) oraz pojedyncze lata.
    - **Wady:** całkowicie ignoruje wyrażenia opisowe. W zdaniu "Lata 90-te były czasem przemian" zwrócił pusty wynik, co dyskwalifikuje go w analizie kontekstu historycznego.

2.  NER
    - **Ocena:** Metoda nieprzydatna do dat.
    - Model poprawnie wykrył lokalizacje, ale w przypadku daty wygenerował tzw. false positive, oznaczając frazę "Pamiętaj o terminie!" jako encję. Nie posiada dedykowanej klasy `DATE`.

3.  LLM
    - **Zalety:** Jako jedyny system poprawnie zinterpretował logikę zakresów "od 2010 do 2015" i okresów "Lata 90-te"* => rok 90.
    - **Wady:** Model ma tendencję do nadgorliwości. W polu `ranges` potrafi umieścić fragmenty tekstu niebędące czasem np. "przeprowadziłem się", "termin". Wymagałoby to dodatkowego czyszczenia danych.

**Strategia Hybrydowa**

W projekcie wykorzystano RegExp do szybkiego indeksowania bazy, natomiast LLM jest używany warunkowo w agencie do interpretacji zapytań użytkownika, gdzie kluczowe jest zrozumienie intencji.

### 4.3.

Zgodnie z wymaganiami, proces wzbogacania danych (Data Enrichment) oraz konfiguracji baz danych został zrealizowany w dwóch krokach:

1. Wzbogacanie danych za pomocą skryptu `enrich_data.py`

  Surowe dane z pliku `culturax_pl_clean.jsonl` zostały przetworzone przez pipeline NLP, który dodał do każdego dokumentu nowe pola:
  - `named_entities`: wynik działania modelu NER.
  - `years`: wynik działania wyrażeń regularnych.

  Przykładowa struktura dokumentu po przetworzeniu:
  json
  {
    "text": "Spotkanie w Warszawie w 2023 roku...",
    "named_entities": ["Warszawie"],
    "years": [2023]
  }

2. Konfiguracja Elasticsearch
  W pliku setup_enriched_db.py zdefiniowano sztywny mapping dla indeksu lab10_hybrid. Jest to kluczowe dla poprawnego filtrowania: 
  - Pole `named_entities` otrzymało typ keyword. Pozwala to na precyzyjne filtrowanie po nazwach własnych.

  - Pole `years` otrzymało typ integer. Pozwala to na wykonywanie zapytań zakresowych, np. years >= 2020.

3. Konfiguracja Qdrant. 

  Do wektorowej bazy danych Qdrant, obok wektora, przesłano payload zawierający te same metadane. Dzięki temu silnik Qdrant może wykonywać tzw. pre-filtering - czyli odrzucać wektory niespełniające kryteriów jeszcze przed obliczeniem podobieństwa cosinusowego, co znacząco przyspiesza wyszukiwanie i zwiększa precyzję.

## 5. NLP w praktyce

In [3]:
import re
import json
import pandas as pd
from rag.retrieval.search_engine import hybrid_search


topics = [
    {
        "name": "Odpowiedzialność",
        "regex_pattern": r"odpowiedzialn\w+",
        "query": "poczucie odpowiedzialności za innych",
        "year_filter": 2020,
        "entity_filter": None 
    },
    {
        "name": "Liderzy",
        "regex_pattern": r"lider\w+|przywódc\w+",
        "query": "cechy dobrego lidera i przywódcy",
        "year_filter": None,
        "entity_filter": "Polsce" 
    },
    {
        "name": "Zaufanie",
        "regex_pattern": r"zaufani\w+|ufnoś\w+",
        "query": "utrata zaufania społecznego",
        "year_filter": 2021,
        "entity_filter": None
    }
]

DATA_FILE = "data/culturax_enriched.jsonl"

def run_regex_benchmark(pattern, limit=3):
    results = []
    try:
        with open(DATA_FILE, 'r', encoding='utf-16') as f:
            for line in f:
                if len(results) >= limit: break
                doc = json.loads(line)
                if re.search(pattern, doc['text'], re.IGNORECASE):
                    results.append(doc)
    except Exception as e:
        print(f"Błąd RegExp: {e}")
    return results

benchmark_data = []

print("Rozpoczynam Benchmark metod NLP.")

for t in topics:
    print(f"\nTemat: {t['name']}.")
    
    hits_regex = run_regex_benchmark(t['regex_pattern'])
    benchmark_data.append({
        "Temat": t['name'], "Metoda": "1. RegExp", 
        "Wynik": hits_regex[0]['text'][:80] + "..." if hits_regex else "BRAK",
        "Metadane": str(hits_regex[0].get('years')) if hits_regex else "-"
    })

    hits_bm25 = hybrid_search(t['query'], filters=None, limit=3, weight_es=1.0, weight_qdrant=0.0)
    benchmark_data.append({
        "Temat": t['name'], "Metoda": "2. BM25", 
        "Wynik": hits_bm25[0]['text'][:80] + "..." if hits_bm25 else "BRAK",
        "Metadane": str(hits_bm25[0].get('years')) if hits_bm25 else "-"
    })

    hits_vec = hybrid_search(t['query'], filters=None, limit=3, weight_es=0.0, weight_qdrant=1.0)
    benchmark_data.append({
        "Temat": t['name'], "Metoda": "3. Vector", 
        "Wynik": hits_vec[0]['text'][:80] + "..." if hits_vec else "BRAK",
        "Metadane": str(hits_vec[0].get('years')) if hits_vec else "-"
    })

    hits_hybrid = hybrid_search(t['query'], filters=None, limit=3)
    benchmark_data.append({
        "Temat": t['name'], "Metoda": "4. Hybrid", 
        "Wynik": hits_hybrid[0]['text'][:80] + "..." if hits_hybrid else "BRAK",
        "Metadane": str(hits_hybrid[0].get('years')) if hits_hybrid else "-"
    })

    filters = {}
    if t['year_filter']:
        filters['years'] = [y for y in range(t['year_filter'], 2025)]
    if t['entity_filter']:
        filters['named_entities'] = [t['entity_filter']]
        
    hits_meta = hybrid_search(t['query'], filters=filters, limit=3)
    benchmark_data.append({
        "Temat": t['name'], "Metoda": "5. Hybrid + METADATA", 
        "Wynik": hits_meta[0]['text'][:80] + "..." if hits_meta else "BRAK",
        "Metadane": str(hits_meta[0].get('years')) if hits_meta else "-"
    })

df_results = pd.DataFrame(benchmark_data)
display(df_results)

Rozpoczynam Benchmark metod NLP.

Temat: Odpowiedzialność.

Temat: Liderzy.

Temat: Zaufanie.


,Temat,Metoda,Wynik,Metadane
0,Odpowiedzialność,1. RegExp,"Za reżyserię filmu ""Kobieta w czerni 2"" jest odpowiedzialny Tom Harper (""Peaky B...",[]
1,Odpowiedzialność,2. BM25,"Odpowiedzialność: Samostanowienie rozwija autonomię, co wiąże się z przejmowanie...",[]
2,Odpowiedzialność,3. Vector,"Odpowiedzialność: Samostanowienie rozwija autonomię, co wiąże się z przejmowanie...",[]
3,Odpowiedzialność,4. Hybrid,"Odpowiedzialność: Samostanowienie rozwija autonomię, co wiąże się z przejmowanie...",[]
4,Odpowiedzialność,5. Hybrid + METADATA,Redakcja Tweet Telegram WhatsApp Email Like this: Like Loading... Mariusz Matusz...,[2022]
5,Liderzy,1. RegExp,"Kiedy osoba jest silnym przywódcą, te wewnętrzne cechy są rozszerzone na innych,...",[]
6,Liderzy,2. BM25,Rozpoznanie kompetencji Kompetentni przywódcy z powodzeniem spełniają wyzwanie w...,[]
7,Liderzy,3. Vector,Rozpoznanie kompetencji Kompetentni przywódcy z powodzeniem spełniają wyzwanie w...,[]
8,Liderzy,4. Hybrid,Rozpoznanie kompetencji Kompetentni przywódcy z powodzeniem spełniają wyzwanie w...,[]
9,Liderzy,5. Hybrid + METADATA,"Dostrzegając ważność i ciągłą aktualność tych tematów, podjęliśmy dyskusję na te...",[]


#### Wyniki

### 5. Raport z Benchmarku NLP

Poniższa tabela przedstawia analizę wyników uzyskanych na korpusie `CulturaX` dla trzech pojęć abstrakcyjnych.

| Temat | Metoda | Sensowność wyniku | Typowe błędy | Czy encje / daty pomogły? |
| :--- | :--- | :--- | :--- | :--- |
| **Odpowiedzialność** | **RegExp** | **Niska** | Całkowity brak kontekstu. Złapano reżysera filmu "odpowiedzialny Tom Harper", co nie ma związku z pojęciem abstrakcyjnym. | Nie |
| | **Vector / Hybrid** | **Wysoka** | Poprawne dopasowanie semantyczne ("Samostanowienie rozwija autonomię"). Model zrozumiał kontekst psychologiczny/socjologiczny. | Nie |
| | **Hybrid + Meta** | **Bardzo Dobra** | Wynik jest poprawny semantycznie i pochodzi z roku **2022**, co gwarantuje aktualność informacji. | **TAK** - wyeliminowano stare teksty. |
| **Liderzy** | **RegExp** | **Średnia** | Złapano ogólne zdanie o przywódcy. Ryzyko złapania "lidera cen". | Nie |
| | **Vector** | **Wysoka** | Trafne dopasowanie "rozpoznanie kompetencji", "kompetentni przywódcy". | Nie |
| | **Hybrid + Meta** | **Wysoka** | Wynik dotyczy dyskusji/panelu, co sugeruje kontekst zgodny z filtrem np. wydarzenie w Polsce. | **TAK** - kontekst lokalny. |
| **Zaufanie** | **Vector** | **Średnia** | Model skojarzył "zaufanie społeczne" z instytucjami, co jest luźnym dopasowaniem, ale nie do końca tym, o co pytano. | Nie |
| | **Hybrid + Meta** | **Techniczna** | Znaleziono newsa z 2021 roku. System rygorystycznie przestrzegał daty, nawet jeśli temat zszedł na newsy gospodarcze. | **TAK** - filtr czasu zadziałał bezbłędnie. |

#### Wnioski

1.  RegExp to "ślepe narzędzie"

    Przypadek "odpowiedzialności" pokazuje, że szukanie po rdzeniu słowa w tematach abstrakcyjnych generuje duży szum informacyjny.

2.  Przewaga wektorów 

    Metody oparte na embeddingach znacznie lepiej radzą sobie z wyłapaniem sensu definicji i pojęć psychologicznych autonomia, kompetencje.

3.  Siła metadanych 

    Dodanie filtrów (rok, encja) działa jak twardy bezpiecznik. Nawet jeśli wyszukiwarka wektorowa chce zwrócić lepszy, ale stary wynik - system wymusza nowsze dane. Jest to kluczowe w systemach biznesowych i newsowych.

### 5.2

In [4]:
import json
import random
from collections import Counter
from rag.retrieval.search_engine import hybrid_search

ALL_DOCS = []
with open("data/culturax_enriched.jsonl", 'r', encoding='utf-16') as f:
    for line in f:
        ALL_DOCS.append(json.loads(line))

def generate_benchmark_dataset():
    topics = [
        "odpowiedzialność zespołowa",
        "przywództwo i liderzy",
        "zaufanie i jego utrata"
    ]
    
    dataset = []
    stats = {t: {"rel_years": [], "rel_ents": [], "noise_years": [], "noise_ents": []} for t in topics}

    print("Generowanie datasetu testowego.")

    for topic in topics:

        relevant_docs = hybrid_search(topic, limit=10)
        
        relevant_ids = set()
        for doc in relevant_docs:
            doc_id = doc.get('id', str(random.randint(10000, 99999))) 
            relevant_ids.add(doc_id)
            
            dataset.append({
                "id": doc_id,
                "text": doc.get('text', '')[:200] + "...", 
                "label": "relevant",
                "topic": topic
            })
            
            stats[topic]["rel_years"].extend(doc.get('years', []))
            stats[topic]["rel_ents"].extend(doc.get('named_entities', []))

        noise_count = 0
        while noise_count < 10:
            candidate = random.choice(ALL_DOCS)
            c_id = candidate.get('id', str(random.randint(10000, 99999)))
            
            if c_id not in relevant_ids:
                dataset.append({
                    "id": c_id,
                    "text": candidate.get('text', '')[:200] + "...",
                    "label": "noise",
                    "topic": topic
                })
                noise_count += 1
                
                stats[topic]["noise_years"].extend(candidate.get('years', []))
                stats[topic]["noise_ents"].extend(candidate.get('named_entities', []))

    return dataset, stats

benchmark_ds, benchmark_stats = generate_benchmark_dataset()

print(f"\ Wygenerowano {len(benchmark_ds)} rekordów.")
print("Przykładowe 2 rekordy:")
print(json.dumps(benchmark_ds[:2], indent=2, ensure_ascii=False))

print("\nAnaliza metadanych:")
for topic, data in benchmark_stats.items():
    print(f"\nTemat: {topic}")
    
    top_rel_years = Counter(data['rel_years']).most_common(3)
    top_noise_years = Counter(data['noise_years']).most_common(3)
    
    top_rel_ents = Counter(data['rel_ents']).most_common(3)
    top_noise_ents = Counter(data['noise_ents']).most_common(3)
    
    print(f"   [Trafne] Typowe lata: {top_rel_years} | Typowe encje: {top_rel_ents}")
    print(f"   [Szum]   Typowe lata: {top_noise_years} | Typowe encje: {top_noise_ents}")

Generowanie datasetu testowego.
\ Wygenerowano 60 rekordów.
Przykładowe 2 rekordy:
[
  {
    "id": "69766",
    "text": "Przedmiotem ubezpieczenia jest odpowiedzialność cywilna adwokata za szkody wyrządzone w następstwie działania lub zaniechania podczas wykonywania czynności zawodowych....",
    "label": "relevant",
    "topic": "odpowiedzialność zespołowa"
  },
  {
    "id": "85147",
    "text": "Odpowiedzialność: Samostanowienie rozwija autonomię, co wiąże się z przejmowaniem odpowiedzialności za wybory, a to prowadzi do jeszcze większej odpowiedzialności....",
    "label": "relevant",
    "topic": "odpowiedzialność zespołowa"
  }
]

Analiza metadanych:

Temat: odpowiedzialność zespołowa
   [Trafne] Typowe lata: [(2011, 1)] | Typowe encje: [('Samostanowienie', 2), ('Tomasz Grzyb', 2), ('European Commission', 1)]
   [Szum]   Typowe lata: [(2002, 1)] | Typowe encje: [('Archidiecezja Glasgow', 1), ('Szkocji', 1), ('Mario Conti', 1)]

Temat: przywództwo i liderzy
   [Trafne] Typowe lata

#### Wyniki

Na podstawie zebranych statystyk można zauważyć kluczowe różnice.

1. Encje:

    Relevant: w dokumentach na tematy "odpowiedzialność" i "zaufanie" pojawiają się encje związane z organizacjami międzynarodowymi np. European Commission, technologią np. DAO - organizacje zaufania cyfrowego oraz pojęciami abstrakcyjnymi błędnie zaklasyfikowanymi jako nazwy własne np. samostanowienie.

    Noise: w dokumentach nietrafnych zdecydowanie dominują konkretne lokalizacje geograficzne niezwiązane z tematem np. Szkocja, Archidiecezja Glasgow, Nadrenia Północna-Westfalia, Tatry. Są to często artykuły newsowe lub turystyczne, które przypadkowo zawierały poszukiwane słowo.

2. Lata (Years):

    W analizowanych próbkach dokumenty trafne często dotyczą lat nowszych np. 2020 w kontekście przywództwa/zaufania, co może wiązać się z publikacjami na temat kryzysów lub pandemii.

    Dokumenty typu "szum" rozkładają się losowo, często wskazując na lata historyczne lub nieistotne dla bieżącej analizy np. 2002, 2006.

Istnieje silna korelacja semantyczna w metadanych. Wykluczenie dokumentów zawierających encje typowo geograficzne gdy nie szukamy lokalizacji oraz filtrowanie po roku pozwala skutecznie odsiać szum informacyjny.

## 6. RAG 

W ostatnim etapie połączono wszystkie moduły w jeden spójny system RAG.

Wybrany problem badawczy:
- "Jak zmienia się narracja o przywództwie i liderach w tekstach opublikowanych po roku 2020?"

Schemat Pipeline'u:

```mermaid
flowchart TD
    User[Użytkownik] -->|Pytanie: 'Liderzy po 2020'| Brain[Agent Brain: Ekstrakcja]
    
    subgraph Metadata Extraction
    Brain -->|LLM/Regex| Filters{Filtry?}
    Filters -->|Lata >= 2020| DateFilter[Filtr Dat]
    Filters -->|Temat: 'Liderzy'| Query[Query Vector]
    end
    
    subgraph Retrieval Layer
    DateFilter & Query --> Hybrid[Hybrid Search Engine]
    Hybrid -->|ES + Qdrant| Docs[Dokumenty Relevant]
    end
    
    subgraph Generation Layer
    Docs -->|Kontekst| Prompt[Prompt RAG]
    Prompt -->|Ollama: Gemma2| Answer[Finalna Odpowiedź]
    end
    
    Answer --> User

### 6.1 Zadanie – QUALITY LOOP

In [5]:
import pandas as pd
from rag.retrieval.search_engine import hybrid_search

test_cases = [
    {"query": "Wydarzenia i kryzysy w 2020 roku", "years": [2020], "entities": []},
    {"query": "Technologia przed rokiem 2015", "years": [2010, 2011, 2012, 2013, 2014], "entities": []},
    {"query": "Przyszłość i plany na 2023", "years": [2023], "entities": []},
    
    {"query": "Sytuacja gospodarcza w Polsce", "years": [], "entities": ["Polsce"]},
    {"query": "Działania Unii Europejskiej", "years": [], "entities": ["Unii Europejskiej"]},
    {"query": "Co słychać w Warszawie?", "years": [], "entities": ["Warszawie"]},
    
    {"query": "Definicja zaufania społecznego", "years": [], "entities": []},
    {"query": "Cechy dobrego lidera", "years": [], "entities": []},
    
    {"query": "Sprzedaż samochodów i rynki", "years": [2018], "entities": []}, 
    {"query": "Bezpieczeństwo danych osobowych", "years": [], "entities": []}
]

results_quality = []

print("Uruchamiam Quality Loop.")

for i, case in enumerate(test_cases):
    q = case['query']
    filters = {}
    if case['years']: filters['years'] = case['years']
    if case['entities']: filters['named_entities'] = case['entities']
    
    docs = hybrid_search(q, filters=filters, limit=1) 
    top_doc = docs[0] if docs else None
    
    if top_doc:
        doc_text = top_doc.get('text', '')[:100] + "..."
        doc_years = str(top_doc.get('years', []))
        doc_ents = str(top_doc.get('named_entities', [])[:3]) 
    else:
        doc_text = "Brak wyników"
        doc_years = "-"
        doc_ents = "-"

    results_quality.append({
        "Nr": i+1,
        "Zapytanie": q,
        "Znaleziony Tekst (Top 1)": doc_text,
        "Metadane (Lata)": doc_years,
        "Metadane (Encje)": doc_ents,
        "Wymagany filtr": f"Lata: {case['years']} | Encje: {case['entities']}"
    })

df_q = pd.DataFrame(results_quality)
pd.set_option('display.max_colwidth', None)
display(df_q)

Uruchamiam Quality Loop.


,Nr,Zapytanie,Znaleziony Tekst (Top 1),Metadane (Lata),Metadane (Encje),Wymagany filtr
0,1,Wydarzenia i kryzysy w 2020 roku,Wydział I Nauk Humanistycznych i Społecznych W roku 2020 Wydział I przyznał następujące nagrody i wy...,[2020],"['Wydział I', 'Wydział I Nauk Humanistycznych i Społecznych']",Lata: [2020] | Encje: []
1,2,Technologia przed rokiem 2015,"O Biznes-Host.pl RobimySEO zaczął obserwować Biznes-Host.pl Grudzień 29, 2015 Biznes-Host.pl zaczął ...","[2012, 2015]","['Biznes - Host', '##ty', 'Biznes']","Lata: [2010, 2011, 2012, 2013, 2014] | Encje: []"
2,3,Przyszłość i plany na 2023,Brak wyników,-,-,Lata: [2023] | Encje: []
3,4,Sytuacja gospodarcza w Polsce,"Konfederacja Lewiatan – najbardziej wpływowa polska organizacja biznesowa, reprezentująca interesy p...",[],"['Unii Europejskiej', 'Konfederacja Lewiatan', 'Polsce']",Lata: [] | Encje: ['Polsce']
4,5,Działania Unii Europejskiej,"Konfederacja Lewiatan – najbardziej wpływowa polska organizacja biznesowa, reprezentująca interesy p...",[],"['Unii Europejskiej', 'Konfederacja Lewiatan', 'Polsce']",Lata: [] | Encje: ['Unii Europejskiej']
5,6,Co słychać w Warszawie?,Wdrożenie ustawy PAD - Andrzej Lanc - 27 czerwca 2018 w Warszawie Rekomendacja D-SKOK Standardy wewn...,[2018],"['Grzegorz Banaś', 'Andrzej Lanc', 'Warszawie']",Lata: [] | Encje: ['Warszawie']
6,7,Definicja zaufania społecznego,"Definicja człowieczeństwa skupia się w pojęciu homo sapiens – człowieka myślącego, który realizuje s...",[],['homo sapiens'],Lata: [] | Encje: []
7,8,Cechy dobrego lidera,Mają one dotyczyć osób zaangażowanych w skazanie na pobyt w kolonii karnej lidera rosyjskiej opozycj...,[],['Aleksieja Nawalnego'],Lata: [] | Encje: []
8,9,Sprzedaż samochodów i rynki,Obecnie zakaz palenia e papierosów obowiązuje... 2018-03-23|Kategoria: Sprzedaż Interntowa / Inne Sk...,"[2017, 2018]","['Inne Sklepy', 'Inne Sklepy Majic Paints', '##rzedaż Interntowa']",Lata: [2018] | Encje: []
9,10,Bezpieczeństwo danych osobowych,"Administrator danych osobowych Administratorem Twoich danych osobowych jest ""Eversun"" Dawid Sipowicz...",[],"['Sopot', 'Dawid Sipowicz']",Lata: [] | Encje: []


#### Wyniki

| Zapytanie | Label | Encje | Daty | Halucynacje / Uwagi | Ocena (0-2) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **1. Wydarzenia 2020** | **Relevant** | *Wydział I* | *[2020]* | Brak. System precyzyjnie znalazł raport z roku 2020. | **2** |
| **2. Technologia < 2015** | **Relevant** | *Biznes-Host* | *[2012, 2015]* | Dokument zawiera rok 2012, choć w tekście pojawia się też 2015. Temat pasuje. | **2** |
| **3. Przyszłość 2023** | **Brak** | *-* | *-* | **Brak wyników.** Korpus `CulturaX` kończy się przed 2023 rokiem, więc system słusznie nic nie wymyślił. | **-** |
| **4. Gospodarka (Polska)** | **Relevant** | *Polsce, Konfederacja Lewiatan* | *-* | Idealne trafienie. Encja "Polsce" oraz organizacja biznesowa. | **2** |
| **5. Unia Europejska** | **Relevant** | *Unii Europejskiej* | *-* | To samo co wyżej - encja została poprawnie wykorzystana do znalezienia kontekstu. | **2** |
| **6. Warszawa** | **Relevant** | *Warszawie* | *[2018]* | Znaleziono newsa o ustawie/wydarzeniu w Warszawie. Kontekst geograficzny zachowany. | **2** |
| **7. Definicja zaufania** | **Noise** | *homo sapiens* | *-* | **Błąd semantyczny.** System znalazł "definicję człowieczeństwa" zamiast "zaufania". Wektory były zbyt blisko ogólnych pojęć filozoficznych. | **1** |
| **8. Cechy lidera** | **Noise** | *Aleksieja Nawalnego* | *-* | **Błąd kontekstu.** Znaleziono słowo "lider", ale w kontekście politycznym/karnym, a nie cech przywódczych. Klasyczny problem wieloznaczności słów. | **1** |
| **9. Sprzedaż aut 2018** | **Noise** | *Inne Sklepy* | *[2018]* | Zgodność roku i słowa "sprzedaż", ale temat dotyczy e-papierosów, a nie aut. | **1** |
| **10. Bezpieczeństwo danych** | **Relevant** | *Eversun* | *-* | Trafienie w klauzulę RODO. | **2** |

**Podsumowanie**
- **Skuteczność filtrów:** mechanizm `hybrid_search` bezbłędnie egzekwuje filtry. Nie zwrócono dokumentów z błędnego roku ani z błędną encją. W przypadku braku danych (rok 2023) system uczciwie zwrócił pusty wynik.
- **Problemy semantyczne:** Wyszukiwanie wektorowe przy pojęciach abstrakcyjnych ma tendencję do kierowania w stronę newsów politycznych lub innych definicji. Wymagałoby to zastosowania rerankera w kolejnym etapie projektu.

#### 6.2 Zadanie – Memory as Knowledge Accumulation

In [6]:
from rag.reasoning.memory import KnowledgeMemory

memory = KnowledgeMemory()


print(" Krok 1: Użytkownik pyta o brakujące dane.")
memory.add_pending(
    query="Jakie są prognozy PKB na 2025 rok?", 
    filters={"years": [2025], "named_entities": ["PKB"]}
)
print("Krok 2: Wpada nowy dokument.")
incoming_doc = {
    "text": "Wzrost PKB w 2025 roku w Polsce prognozowany jest na poziomie 3%...",
    "years": [2025],
    "named_entities": ["PKB", "Polska"]
}

triggered = memory.check_new_document(incoming_doc)

if triggered:
    print("\nPowiadomienie dla użytkownika:")
    for t in triggered:
        print(f"   Znaleźliśmy nowe informacje dla Twojego pytania: {t['query']}'")

 Krok 1: Użytkownik pyta o brakujące dane.
Zapisano intencję: 'Jakie są prognozy PKB na 2025 rok?' Czekam na dane: {'years': [2025], 'named_entities': ['PKB']}
Krok 2: Wpada nowy dokument.
Nowy dokument: 'Wzrost PKB w 2025 roku w Polsce prognozo.' [Rok: [2025]]
   Dokument pasuje do oczekującego pytania: 'Jakie są prognozy PKB na 2025 rok?'.

Powiadomienie dla użytkownika:
   Znaleźliśmy nowe informacje dla Twojego pytania: Jakie są prognozy PKB na 2025 rok?'


#### Wyniki

Na podstawie powyższych logów system zadziałał poprawnie w modelu Event-Driven:
1.  Rejestracja braku wiedzy: system poprawnie zidentyfikował brak danych dla zapytania o rok 2025 i zamiast halucynować, zapisał intencję w pamięci (`pending_queries`).
2.  Dopasowanie: w momencie nadejścia nowego dokumentu (zawierającego rok `2025` i encję `PKB`), mechanizm `check_new_document` skutecznie powiązał go z oczekującym zapytaniem.
3.  Reakcja: system automatycznie wyzwolił akcję powiadomienia, zamykając pętlę "Quality Loop".


#### Odpowiedzi na pytania

1. Jak system powinien obchodzić się z nowymi dokumentami?
    System nie powinien przeszukiwać całej bazy historycznej przy każdym nowym dokumencie (byłoby to nieefektywne). Zamiast tego należy zastosować **Ingestion Hook**:
    - Każdy nowy dokument wpadający do pipeline'u (np. z crawlera) jest "skanowany" w locie.
    - Wyciągamy z niego metadane (Lata, Encje).
    - Porównujemy je z "lekką" listą oczekujących zapytań (Pending Queries) przechowywaną w szybkiej pamięci (np. Redis).
    - Jeśli metadane się pokrywają (np. *Dokument ma rok 2024* AND *Oczekujące zapytanie chce rok 2024*), oznaczamy zapytanie jako "gotowe do ponowienia".

2. Jak RAG mógłby „przypomnieć sobie” o starych zapytaniach?
    Proces przypominania powinien być asynchroniczny, aby nie blokować indeksowania danych:
    1.  Zmiana Stanu: gdy Ingestion Hook wykryje dopasowanie, zmienia status zapytania z `waiting` na `retry_ready`.
    2.  Worker: ssobny proces działający w tle lub okresowo jako CRON pobiera zapytania o statusie `retry_ready`.
    3.  Weryfikacja: worker wykonuje pełne wyszukiwanie RAG z użyciem nowego dokumentu. Jeśli Confidence Score odpowiedzi jest wysoki, uznaje problem za rozwiązany.

3. Czy system powinien powiadomić użytkownika?
    Zdecydowanie tak. To kluczowa cecha odróżniająca inteligentnego agenta od zwykłej wyszukiwarki.
    - Kanał: e-mail, powiadomienie push lub wiadomość na czacie w zależności od interfejsu.
    - Format: "Cześć! Pytałeś ostatnio o [Temat]. Właśnie przetworzyliśmy nowy raport, który zawiera odpowiedź na Twoje pytanie: [Link/Odpowiedź]."
    - Wartość: buduje to zaufanie użytkownika do systemu, który "pamięta" i "pracuje w tle", nawet gdy użytkownik nie jest aktywny.

### 6.3 Zadania związane z promptami dla NER i dat

In [7]:
import json
import requests
import re
from rag.metadata.ner import NERExtractor

ner_model = NERExtractor()

def extract_with_prompt(text, prompt_type="simple"):
    if prompt_type == "simple":
        system_prompt = """
        Wyodrębnij z tekstu wszystkie nazwane encje:
        - persons: Imię i nazwisko
        - organizations: Firmy, instytucje
        - locations: Miasta, kraje
        
        Zwróć TYLKO JSON w formacie:
        { "persons": [], "organizations": [], "locations": [] }
        """
    else:
        system_prompt = """
        Jesteś ekspertem od analizy przywództwa. Z tekstu wyodrębnij:
        - leaders: Osoby pełniące rolę przywódczą np. prezesi, dyrektorzy, liderzy opinii.
        - organizations: Organizacje, którymi zarządzają.
        - locations: Miejsca kluczowe dla kontekstu np. siedziba, miejsce konferencji.
        
        Ignoruj osoby, które nie są liderami np. zwykli pracownicy, świadkowie.
        
        Zwróć TYLKO JSON w formacie:
        { "leaders": [], "organizations": [], "locations": [] }
        """

    full_prompt = f"{system_prompt}\n\nTekst: {text}"
    
    payload = {
        "model": "gemma2:2b",
        "messages": [{"role": "user", "content": full_prompt}],
        "stream": False,
        "options": {"temperature": 0.0}
    }
    
    try:
        response = requests.post("http://localhost:11434/api/chat", json=payload, timeout=60)
        content = response.json()['message']['content']
        clean_json = re.sub(r'```json|```', '', content).strip()
        return json.loads(clean_json)
    except Exception as e:
        return {"error": str(e)}

sample_text = """
Podczas szczytu w Davos, Satya Nadella, CEO Microsoftu, ogłosił nową strategię AI. 
W kuluarach rozmawiał z nim Jan Kowalski, analityk z Warszawy, który przyglądał się zmianom.
"""

print(f"Tekst: {sample_text.strip()}\n")

print("1. Model NER")
print(ner_model.extract(sample_text))

print("\n2. LLM - wariat prosty")
print(json.dumps(extract_with_prompt(sample_text, "simple"), indent=2, ensure_ascii=False))

print("\n3. LLM wariant skomplikowany")
print(json.dumps(extract_with_prompt(sample_text, "contextual"), indent=2, ensure_ascii=False))

Ładowanie modelu NER: Babelscape/wikineural-multilingual-ner.


Device set to use cpu


Model NER gotowy.
Tekst: Podczas szczytu w Davos, Satya Nadella, CEO Microsoftu, ogłosił nową strategię AI. 
W kuluarach rozmawiał z nim Jan Kowalski, analityk z Warszawy, który przyglądał się zmianom.

1. Model NER
['Jan Kowalski', 'Warszawy', 'Davos', 'Satya Nadella', 'Microsoft']

2. LLM - wariat prosty
{
  "persons": [
    "Satya Nadella",
    "Jan Kowalski"
  ],
  "organizations": [
    "Microsoftu"
  ],
  "locations": [
    "Davos",
    "Warszawa"
  ]
}

3. LLM wariant skomplikowany
{
  "leaders": [
    "Satya Nadella"
  ],
  "organizations": [
    "Microsoftu"
  ],
  "locations": [
    "Davos"
  ]
}


#### Wyniki

1. NER 

    Jest "zachłanny". Złapał wszystko - Jana i Satyę - ale wrzucił ich do jednego worka. Dla modelu BERT Jan Kowalski i CEO Microsoftu to po prostu PER.

2. LLM prosty

    Zrobił to samo co NER, ale ładnie posortował na kategorie. Zamienił "Warszawy" na mianownik "Warszawa" czego zwykły NER często nie robi bez lemmatyzatora.

3. LLM kontekstowy 

    Zrozumiał, że Jan Kowalski nie jest liderem, tylko analitykiem, i wyrzucił go z listy leaders. Zrozumiał też, że Warszawa nie jest kluczowa dla kontekstu przywództwa w tym zdaniu - nie była miejscem szczytu, tylko pochodzenia analityka - więc ją pominął.

### 6.4 Prompt dla dat w kontekście RAG

In [8]:
import json
import requests
import re

def analyze_date_relevance(text, question):

    prompt = f"""
    Jesteś analitykiem czasu w dokumentach. Twoim zadaniem jest:
    1. Wyodrębnić wszystkie daty z poniższego tekstu.
    2. Określić, czy te daty są istotne dla odpowiedzi na pytanie użytkownika.
    
    Pytanie użytkownika: "{question}"
    
    Tekst do analizy: "{text}"
    
    Zwróć TYLKO JSON w formacie:
    {{
      "extracted_dates": ["YYYY-MM-DD", "YYYY", ...],
      "is_relevant": true/false,
      "explanation": "Krótkie uzasadnienie dlaczego pasuje lub nie."
    }}
    """
    
    payload = {
        "model": "gemma2:2b",
        "messages": [{"role": "user", "content": prompt}],
        "stream": False,
        "options": {"temperature": 0.0}
    }
    
    try:
        response = requests.post("http://localhost:11434/api/chat", json=payload, timeout=60)
        content = response.json()['message']['content']
        clean_json = re.sub(r'```json|```', '', content).strip()
        return json.loads(clean_json)
    except Exception as e:
        return {"extracted_dates": [], "is_relevant": True, "explanation": "Error: " + str(e)}

question = "Jakie zmiany w podatkach wprowadzono w 2022 roku?"

retrieved_docs = [
    {
        "id": 1, 
        "text": "W 2015 roku wprowadzono ulgę na dzieci, która zmieniła system podatkowy.",
        "base_score": 0.95  
    },
    {
        "id": 2, 
        "text": "Nowy Ład podatkowy wchodzi w życie od stycznia 2022 roku.",
        "base_score": 0.92
    },
    {
        "id": 3, 
        "text": "Ministerstwo Finansów planuje zmiany. (Brak daty)",
        "base_score": 0.80
    }
]

print(f"Pytanie: {question}\n")
print("Rozpoczynam reranking .\n")

reranked_docs = []

for doc in retrieved_docs:
    analysis = analyze_date_relevance(doc['text'], question)
    
    final_score = doc['base_score']
    penalty = 0.0
    
    if not analysis['is_relevant']:
        penalty = 0.5
        final_score -= penalty
        
    doc['analysis'] = analysis
    doc['final_score'] = final_score
    reranked_docs.append(doc)
    
    print(f"   Doc ID: {doc['id']}")
    print(f"   Daty: {analysis.get('extracted_dates')}")
    print(f"   Relevant: {analysis.get('is_relevant')} | Wyjaśnienie: {analysis.get('explanation')}")
    print(f"   Score: {doc['base_score']} -> {final_score:.2f}\n")

reranked_docs.sort(key=lambda x: x['final_score'], reverse=True)

print("Finalny ranking dokumentów:")
for doc in reranked_docs:
    print(f"   [{doc['final_score']:.2f}] Doc {doc['id']}: {doc['text']}")

Pytanie: Jakie zmiany w podatkach wprowadzono w 2022 roku?

Rozpoczynam reranking .

   Doc ID: 1
   Daty: []
   Relevant: False | Wyjaśnienie: W tekście nie ma informacji o zmianach podatkowych w 2022 roku. Tekst dotyczy wprowadzenia ulgi na dzieci w 2015 roku.
   Score: 0.95 -> 0.45

   Doc ID: 2
   Daty: ['2022-01-01']
   Relevant: True | Wyjaśnienie: Data ta jest istotna dla pytania użytkownika, ponieważ zawiera datę w której wprowadzono nowy system podatkowy.
   Score: 0.92 -> 0.92

   Doc ID: 3
   Daty: []
   Relevant: False | Wyjaśnienie: Brak daty w tekście. Pytanie dotyczy konkretnych zmian w podatkach w roku 2022, a tekst nie zawiera informacji o konkretnej dacie.
   Score: 0.8 -> 0.30

Finalny ranking dokumentów:
   [0.92] Doc 2: Nowy Ład podatkowy wchodzi w życie od stycznia 2022 roku.
   [0.45] Doc 1: W 2015 roku wprowadzono ulgę na dzieci, która zmieniła system podatkowy.
   [0.30] Doc 3: Ministerstwo Finansów planuje zmiany. (Brak daty)


#### Wyniki

Mechanizm poprawnie zmienił kolejność dokumentów, naprawiając błędy standardowego wyszukiwania.

1. Eliminacja "starych newsów"
    Dokument o uldze z 2015 roku miał pierwotnie najwyższy wynik (`0.95`) ze względu na idealne dopasowanie słów kluczowych. LLM poprawnie zidentyfikował, że kontekst czasowy jest sprzeczny z pytaniem. Nałożono karę punktową -0.5, degradując dokument na niższą pozycję (`0.45`).

2. Promocja właściwego kontekstu
    Dokument o "Nowym Ładzie" zawierał poszukiwaną datę 2022. LLM oznaczył go jako `Relevant: True`. Dokument awansował na 1. miejsce w rankingu (`0.92`), stając się głównym źródłem dla odpowiedzi.

3. Surowa ocena braku danych
    Dokument bez dat został oceniony jako nieistotny dla tak precyzyjnego zapytania. Pokazuje to, że LLM potrafi odróżnić ogólniki od konkretów.

#### Wnioski

Zastosowanie LLM do weryfikacji metadanych na etapie rerankingu pozwala uniknąć typowego błędu systemów RAG, czyli generowania odpowiedzi na podstawie nieaktualnych, ale leksykalnie podobnych dokumentów.

### 6.5 Finalny prototyp – agent badawczy

In [9]:
from rag.reasoning.research_agent import ResearchAgent

agent = ResearchAgent()



print("Test A: Sukces")
response_success = agent.solve("Co działo się w Warszawie w 2018 roku?")
print(f"\nWynik: {response_success}\n")

print("Test B: Pamięć")
response_memory = agent.solve("Jaka będzie inflacja w Polsce w 2030 roku?")
print(f"\nWynik: {response_memory}\n")

Test A: Sukces

Rozpoczynam proces dla: 'Co działo się w Warszawie w 2018 roku?'
   Filtry: {'years': [2018], 'named_entities': ['Warszawa']}
   Znaleziono 2 kandydatów.
   Tworzę odpowiedź z 2 źródeł.

Wynik: W 2018 roku Warszawa była popularnym miejscem dla inwestorów, którzy planowali zwiększyć swoją aktywność. 


Test B: Pamięć

Rozpoczynam proces dla: 'Jaka będzie inflacja w Polsce w 2030 roku?'
   Filtry: {'years': [2030], 'named_entities': ['Polska', '']}
   Znaleziono 0 kandydatów.
   Brak danych. Zapisuję w pamięci.
Zapisano intencję: 'Jaka będzie inflacja w Polsce w 2030 roku?' Czekam na dane: {'years': [2030], 'named_entities': ['Polska', '']}

Wynik: Nie posiadam obecnie tych informacji. Zapisałem Twoje pytanie i powiadomię Cię, gdy pojawią się nowe dane.



#### Wyniki

Przeprowadzone testy potwierdzają pełną funkcjonalność systemu `ResearchAgent`:

1. Skuteczność filtracji - Test A

    Agent poprawnie wyekstrahował metadane - `2018`, `Warszawa`. Wyszukiwarka hybrydowa znalazła pasujące dokumenty. LLM wygenerował odpowiedź opartą na faktach "Warszawa była popularnym miejscem dla inwestorów...".

2. Autonomia i pamięć - Test B

    Agent rozpoznał zapytanie o przyszłość `2030`, której nie ma w korpusie `CulturaX`. Zamiast generować halucynacje, system przerwał proces generowania na etapie retrieval. Zapytanie zostało automatycznie zapisane w `KnowledgeMemory` z odpowiednimi tagami, co umożliwi powiadomienie użytkownika w przyszłości.


## 7. Wnioski końcowe

### Które procedury zadziałały najlepiej

- Hybrid Search: najlepsza metoda retrievalu. Wektory znajdowały sens, a słowa kluczowe naprawiały precyzję przy nazwach własnych.

- Filtracja twarda: zastosowanie metadanych na poziomie bazy danych dało 100% skuteczność w eliminowaniu dokumentów spoza zakresu czasowego.

- Active Memory: mechanizm odkładania trudnych pytań "na później" okazał się skuteczniejszą strategią niż próba generowania odpowiedzi na siłę.

### Kiedy NER i daty naprawdę pomogły

- Ujednoznacznienie kontekstu: przy zapytaniu o "Warszawę", dodanie encji LOC: Warszawa pozwoliło wyeliminować dokumenty, gdzie słowo to padało przypadkowo np. w adresie siedziby firmy w stopce, a skupić się na tekstach o mieście.

- Podróż w czasie: przy zapytaniu o "podatki w 2022", mechanizm weryfikacji dat uratował system przed podaniem informacji o podatkach z 2015 roku, które były leksykalnie bardzo podobne, ale merytorycznie błędne.

- Odsiewanie szumu: w benchmarku abstrakcyjnym, filtry encji pozwoliły oddzielić artykuły psychologiczne/biznesowe od ogłoszeń i spamu.